In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome()

In [8]:
import pandas as pd
import numpy as np
df = pd.DataFrame()

In [9]:
movies_dict = [
'into_the_wild',
'rush_hour_3',
'avengers_age_of_ultron',
'spiderman_3',
'mad_max_fury_road',
'zootopia',
'ratatouille',
'tomorrowland_2015',
'nightcrawler',
'2012',
'21-jump-street-2011',
'shawshank_redemption',
'1010792-its_a_wonderful_life',
'hugo',
'live_die_repeat_edge_of_tomorrow',
'the_choice',
'wonder_woman_2017',
'10011582-TRON_legacy',
'taken',
'it_2017',
'equilibrium',
'awake',
'300',
'real_steel',
'the_dark_knight_rises',
'pirates_of_the_caribbean_3',
'the_hateful_eight',
'lost_world_jurassic_park',
'10010662-my_sisters_keeper',
'la_la_land',
'scott_pilgrims_vs_the_world',
'donnie_darko',
'the_perks_of_being_a_wallflower',
'godfather_part_ii',
'frozen_2013',
'cars_3',
'wrong_turn',
'1198124-shutter_island',
'italian_job',
'oceans_twelve',
'annabelle_creation',
'the_conjuring_2'
]

FOR CRITIC REVIEWS

In [10]:
### For critic reviews
for i in range(0,len(movies_dict)):
        url = "https://www.rottentomatoes.com" 
        url = url + '/m/' + movies_dict[i] + '/reviews/?type=top_critics'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html,"lxml")
        x = len(soup.find_all(class_ = 'the_review'))
        movie_name = np.repeat(movies_dict[i].replace('_',' '),x)
        top_critic = [soup.find_all(class_= 'the_review')[j].get_text().lstrip().rstrip() for j in range(0,len(soup.find_all(class_ = 'the_review')))]
        d = {'movie_name': movie_name, 'top_critic': top_critic}
        test = pd.DataFrame(data=d)
        df = df.append(test)

FOR AUDIENCE REVIEWS

In [ ]:
for i in range(0,len(movies_dict)):
    for a in range(1,26):
        url = "https://www.rottentomatoes.com" 
        url = url + '/m/' + movies_dict[i] + '/reviews/?page=' + str(a) + '&type=user&sort='
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html,"lxml")
        x = len(soup.find_all(class_ = 'user_review'))
        movie_name = np.repeat(movies_dict[i].replace('_',' '),x)
        top_critic = [soup.find_all(class_= 'user_review')[j].get_text().lstrip().rstrip() for j in range(0,len(soup.find_all(class_ = 'user_review')))]
        d = {'movie_name': movie_name, 'top_critic': top_critic}
        test = pd.DataFrame(data=d)
        df = df.append(test)

FOR MOVIE RATINGS

In [19]:
### for ratings
for i in range(0,len(movies_dict)):
    url = "https://www.rottentomatoes.com" 
    url = url + '/m/' + movies_dict[i]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,"lxml")
    movie_name = movies_dict[i].replace('_',' ')
    critic_rating = soup.find('div',attrs = {'id':'scoreStats'}).text.lstrip().replace('\n','').replace(' ','')[14:17]
    audience_rating = soup.find('div',attrs= {'class':'audience-info hidden-xs superPageFontColor'}).text.lstrip().replace('\n','').replace(' ','')[14:17]
    movie_type = soup.find_all('div',attrs = {'class':'meta-value'})[0].text
    genre = soup.find_all('div',attrs = {'class':'meta-value'})[1].text.replace('\n','').replace(' ','')
    director = soup.find_all('div',attrs = {'class':'meta-value'})[2].text.replace('\n','')
    boxoffice_collection = soup.find_all('div',attrs = {'class':'meta-value'})[6].text.replace('\n','').replace(',','').replace('$','')
    d_values = [(movie_name,critic_rating,audience_rating,movie_type,genre,director,boxoffice_collection)]
    d_labels = ['movie_name','critic_rating','audience_rating','movie_type','genre','director','boxoffice_collection']
    test = pd.DataFrame(d_values,columns=d_labels)
    df = df.append(test)

WORD FREQUENCY

In [ ]:
import pandas as pd
from nltk.corpus import brown
import nltk
import re
import itertools
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
orig_data = pd.read_csv('ratings_critics_review.csv',encoding = 'ISO-8859-1')
orig_data['top_critic']=orig_data['top_critic'].str.lower()
orig_data['top_critic']=orig_data['top_critic'].astype(str)
for i in range(len(orig_data)):    
    words = re.findall(r"\w+",orig_data['top_critic'][i])
    orig_data['top_critic'][i]= set(words)

In [ ]:
orig_data['top_critic'] = orig_data['top_critic'].apply(list)
final = orig_data.groupby('movie_name').agg(lambda x: x.tolist())
final.reset_index(inplace = True)
for i in range(len(final)):
    final['top_critic'][i] = list(itertools.chain(*final['top_critic'][i]))

In [ ]:
stop = stopwords.words('english') + list(string.punctuation)
for j in range(len(final)):
    final['top_critic'][j] = [i for i in final['top_critic'][j] if i not in stop]
    final['top_critic'][j] = list(map(lemmatizer.lemmatize, final['top_critic'][j]))

In [ ]:
final_rev = final.loc[(final['movie_name'] == 'the dark knight rises') | (final['movie_name'] == 'avengers age of ultron')
| (final['movie_name'] == 'wonder woman 2017') | (final['movie_name'] == 'frozen 2013') | (final['movie_name'] == 'zootopia')
| (final['movie_name'] == 'it 2017') | (final['movie_name'] == '300') | (final['movie_name'] == '21-jump-street-2011')
| (final['movie_name'] == 'mad max fury road')  | (final['movie_name'] == '1198124-shutter island')]    


SENTIMENT ANALYZER

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
analyzer = SentimentIntensityAnalyzer()

In [ ]:
critic = pd.read_csv('Critic reviews.csv',encoding = "ISO-8859-1")
critic_rev = critic['Critic reviews']
df = pd.DataFrame()

In [ ]:
for i in range(0,len(critic_rev)):
    sentscore = analyzer.polarity_scores(critic_rev[i])
    review = critic_rev[i]
    #d = {'review': review, 'sentscore': str(sentscore["compound"])}
    d_values = [(review,str(sentscore["compound"]))]
    d_labels = ['review','sentscore']
    test = pd.DataFrame(d_values, columns = d_labels)
    df = df.append(test)